<h1><u>"MODELO de REGRESIóN: Análisis exploratorio de un dataset de precios de propiedades"</h1></u>
<br>
<h2>DATA SCIENCE:</h2>
<ul>
    <li>González Pluchino, Leandro</li>
</ul>

![Imagen](https://www.academiadeinversion.com/wp-content/uploads/2014/03/como-calcular-valor-inmueble-metodos-ejemplos.png)

**Montaje para la unidad en Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importación de las librerías

In [2]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly as pl
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
pl.offline.init_notebook_mode(connected=True)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
pd.options.display.float_format = '{:,}'.format

# Importación del Conjunto de datos

In [5]:
#Para correr en Notebook de manera local
#Creamos el dataframe con la base de Properatti
path="/content/drive/MyDrive/properatti.csv"
df=pd.read_csv(path)
df.head(40)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.0","-34.6618237,-58.5088387",-34.6618237,...,40.0,"1,127.2727272727273","1,550.0",NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,"3,432,039.0","-34.9038831,-57.9643295",-34.9038831,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.0","-34.6522615,-58.5229825",-34.6522615,...,55.0,"1,309.090909090909","1,309.090909090909",NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,"3,431,333.0","-34.6477969,-58.5164244",-34.6477969,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,"3,435,548.0","-38.0026256,-57.5494468",-38.0026256,...,35.0,"1,828.571428571429","1,828.571428571429",NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,"3,433,657.0","-33.0140714,-58.519828",-33.0140714,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,"3,430,511.0","-34.5329567,-58.5217825",-34.53295670000001,...,78.0,"1,226.4150943396226","1,666.666666666667",NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.0","-34.5598729,-58.443362",-34.5598729,...,40.0,"3,066.666666666666","3,450.0",NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.0","-34.5598729,-58.443362",-34.5598729,...,60.0,"3,000.0","3,250.0",NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
9,9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,"3,838,574.0","-32.942031,-60.7259192",-32.942031,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 76

In [6]:
#Para correr Notebook en collab
#Creamos el dataframe con la base de Properatti
#path="/content/drive/MyDrive/data/properatti.csv"
#df=pd.read_csv(path)
#df.head(40)

In [7]:
#Vemos las dimensiones de la base con el metodo shape
#filas=121.220
#columnas=26
df.shape

(121220, 26)

In [8]:
#Listamos los campos del dataframe
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [9]:
#Renombramos los campos
dic_reemplazo={'property_type':'Tipo de propiedad',
               'place_name':'Partido',
               'state_name':'Provincia',
               "price":"Precio Aviso",
               "currency":"Moneda Aviso",
               "surface_covered_in_m2":"Superfecie Cubierta",
               "price_usd_per_m2":"Precio M2 USD",
               "price_per_m2":"Precio M2",
               "floor":"Piso",
               "expenses":"Expensas",
               "rooms":"Ambientes",
               "description":"Descripcion",
               "title":"Título",
               "price_aprox_local_currency":"Precio ML",
               "surface_total_in_m2":"Superficie Total",
               "price_aprox_usd":"Precio USD",
               "Unnamed: 0":"ID de registro",
               "lat":"Latitud",
               "lon":"Longitud",
               "lat-lon": "LatLon",
               "operation":"Operacion",
               "country_name":"Pais"}

df.rename(columns = dic_reemplazo, inplace = True)

#Eliminamos las columnas inncesarias:


In [10]:
#Demostramos que "Unnamed: 0" es igual al indice por lo que resulta redundante
df["index"]=list(df.index) #creamos una nueva columna que contiene el index
(df["index"]==df["ID de registro"]).value_counts()[1] #comparamos "index" con "Unnamed: 0", lo que genera una lista de booleanos
                                                  #y luego contamos los valores verdaderos. Devuelve 121.220 Trues,
                                                  #vemos que hay match en todos los registros de las columnas

121220

In [11]:
#Demostramos que las columnas "operation" y "country_name" contienen un solo valor unico, por lo que no aportan info relevante
df_val_unicos=pd.DataFrame({"unique_values":df.apply(lambda x : x.unique())})
df_val_unicos["q_unique_values"]=df_val_unicos["unique_values"].apply(lambda x: len(x))
df_val_unicos

,unique_values,q_unique_values
ID de registro,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",121220
Operacion,[sell],1
Tipo de propiedad,"[PH, apartment, house, store]",4
Partido,"[Mataderos, La Plata, Liniers, Centro, Gualegu...",1061
place_with_parent_names,"[|Argentina|Capital Federal|Mataderos|, |Argen...",1164
Pais,[Argentina],1
Provincia,"[Capital Federal, Bs.As. G.B.A. Zona Sur, Buen...",28
geonames_id,"[3430787.0, 3432039.0, 3431333.0, 3435548.0, 3...",647
LatLon,"[-34.6618237,-58.5088387, -34.9038831,-57.9643...",47204
Latitud,"[-34.6618237, -34.9038831, -34.6522615, -34.64...",46496


In [12]:
#Forma alternativa para desmotrar lo anterior
df.Pais.describe()

count        121220
unique            1
top       Argentina
freq         121220
Name: Pais, dtype: object

In [13]:
df.Operacion.describe()

count     121220
unique         1
top         sell
freq      121220
Name: Operacion, dtype: object

In [14]:
#Dropeamos las columnas mencionadas
#Tambien dropeamos las columnas que contienen url's ya que no contienen informacion relevante
df=df.drop(["ID de registro","Operacion","index", "Pais", "image_thumbnail", "properati_url"], axis=1)
df.columns

Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'LatLon', 'Latitud', 'Longitud', 'Precio Aviso',
       'Moneda Aviso', 'Precio ML', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Expensas', 'Descripcion', 'Título'],
      dtype='object')

In [15]:
#Analizamos la cantidad de nulls por columna en proporcion al total de datos.
#Las columnas con mas campos incompletos se ubican al final de la tabla
df_not_nulls= df.notnull().sum(axis=0) / len(df)
df_not_nulls.round(4).sort_values(ascending = False)

Tipo de propiedad            1.0
Provincia                    1.0
Descripcion                  1.0
Título                       1.0
place_with_parent_names      1.0
Partido                   0.9998
geonames_id               0.8456
Superfecie Cubierta       0.8358
Precio USD                0.8316
Precio ML                 0.8316
Moneda Aviso              0.8316
Precio Aviso              0.8316
Precio M2                 0.7231
Superficie Total          0.6756
LatLon                    0.5747
Longitud                  0.5747
Latitud                   0.5747
Precio M2 USD             0.5661
Ambientes                 0.3909
Expensas                  0.1177
Piso                      0.0652
dtype: float64

In [16]:
#Las columnas de latitu y longitud presentan el mismo porcentaje de valores nulos
#Verificamos si cada columna tiene valores en los mismos registros que las otras
lat_indexes = list(df['Latitud'].notnull().index)
long_indexes = list(df['Longitud'].notnull().index)
latlong_indexes = list(df['LatLon'].notnull().index)

print((lat_indexes==long_indexes),
(lat_indexes==latlong_indexes),
(long_indexes==latlong_indexes))

#las columnas presentan valores en los mismos registros.
#Dropeamos la columna de LatLon (lat-lon)
df=df.drop(["LatLon"], axis=1)
df.columns

True True True


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio Aviso', 'Moneda Aviso',
       'Precio ML', 'Precio USD', 'Superficie Total', 'Superfecie Cubierta',
       'Precio M2 USD', 'Precio M2', 'Piso', 'Ambientes', 'Expensas',
       'Descripcion', 'Título'],
      dtype='object')

In [17]:
#Notamos que las columnas de precios suelen tener el mismo porcentage de registros con datos.
#Si encontramos un tipo de cambio fijo podriamosmantener unicamante una columna de precio, y eliminar las demas junto con el de tipo de moneda
#Verificamos si cada columna tiene valores en los mismos registros que las otras
Precio_Aviso_indexes = list(df['Precio Aviso'].notnull().index)
Moneda_Aviso_indexes = list(df['Moneda Aviso'].notnull().index)
Precio_ML_indexes = list(df['Precio ML'].notnull().index)
Precio_USD_indexes = list(df['Precio USD'].notnull().index)

print((Precio_Aviso_indexes==Moneda_Aviso_indexes),
(Precio_Aviso_indexes==Precio_ML_indexes),
(Precio_USD_indexes==Precio_Aviso_indexes))

# guardo el tipo de cambio para un analisis posterior
def Tipo_Cambio(arg):
    if (arg["Precio ML"] is None) or (arg["Precio USD"] is None) or (arg["Precio USD"]==0):
        pass
    else:
        TC=arg["Precio ML"]/arg["Precio USD"]
        return TC

df["TC"]=df.apply(Tipo_Cambio, axis=1)

df=df.drop(['Precio Aviso', 'Moneda Aviso', 'Precio ML'], axis=1)
df.columns

True True True


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Expensas', 'Descripcion', 'Título', 'TC'],
      dtype='object')

In [18]:
#expensas
pattern = "(\$|pesos|dolares|Pesos|Dolares|ARS|USD|PEN|UYU)(\s)*([0-9]+)|([0-9]+)(\s)*(\$|pesos|dolares|Pesos|Dolares|ARS|USD|PEN|UYU)"
regex = re.compile(pattern)

result = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
df_result = pd.DataFrame(result.apply(lambda x : len(x) > 0))

#cantidad de pisos que pueden ser recuperados en la descripcion
expenses_in_descriptions = len(result[df_result['Descripcion']]) / len(df)
expenses_in_descriptions

#Notamos que la cantidad de valores relacionados a precios en la descripcion no es significativa como para salvar la columna
df=df.drop(["Expensas"], axis=1)
df.columns

Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Descripcion', 'Título', 'TC'],
      dtype='object')

In [19]:
# recontruccion de informacion en base a la descripcion
#pisos
#pattern = "piso\s([0-9]+)|([a-z]+)\spiso|piso\s([a-z]+)|([0-9]+)([a-z]+)\spiso"
pattern_1 = "([0-9]+)(\s)*(piso|floor)"
#pattern_2 = "(([a-z]+))(\s)*(piso|floor)"
#pattern_3 = "(piso|floor)(\s)*([0-9]+)"
#pattern_4 = "(piso|floor)(\s)*(([a-z]+))"
#pattern_5 = "planta(\s)*(baja)*([0-9]+)*"
#pattern_6 = "piso|planta|floor"
regex_1 = re.compile(pattern_1)
#regex_2 = re.compile(pattern_2)
#regex_3 = re.compile(pattern_3)
#regex_4 = re.compile(pattern_4)
#regex_5 = re.compile(pattern_5)
#regex_6 = re.compile(pattern_6)

result_1 = df['Descripcion'].apply(lambda x: regex_1.findall(str(x)))
#result_2 = df['Descripcion'].apply(lambda x: regex_2.findall(str(x)))
#result_3 = df['Descripcion'].apply(lambda x: regex_3.findall(str(x)))
#result_4 = df['Descripcion'].apply(lambda x: regex_4.findall(str(x)))
#result_5 = df['Descripcion'].apply(lambda x: regex_5.findall(str(x)))
#result_6 = df['Descripcion'].apply(lambda x: regex_6.findall(str(x)))

df_result_1 = pd.DataFrame(result_1.apply(lambda x : len(x) > 0))
#df_result_2 = pd.DataFrame(result_2.apply(lambda x : len(x) > 0))
#df_result_3 = pd.DataFrame(result_3.apply(lambda x : len(x) > 0))
#df_result_4 = pd.DataFrame(result_4.apply(lambda x : len(x) > 0))
#df_result_5 = pd.DataFrame(result_5.apply(lambda x : len(x) > 0))
#df_result_6 = pd.DataFrame(result_6.apply(lambda x : len(x) > 0))

#cantidad de pisos que pueden ser recuperados en la descripcion
floors_in_descriptions_1 = len(result_1[df_result_1['Descripcion']]) / len(df)
#floors_in_descriptions_2 = len(result_2[df_result_2['Descripcion']]) / len(df)
#floors_in_descriptions_3 = len(result_3[df_result_3['Descripcion']]) / len(df)
#floors_in_descriptions_4 = len(result_4[df_result_4['Descripcion']]) / len(df)
#floors_in_descriptions_5 = len(result_5[df_result_5['Descripcion']]) / len(df)
#floors_in_descriptions_6 = len(result_6[df_result_6['Descripcion']]) / len(df)

# vemos que si sumamos la cantidad de registros que potencialmente podrian recuperarse no llegamos a una cantidad de datos relevantes
print(floors_in_descriptions_1)
#print(floors_in_descriptions_2)
#print(floors_in_descriptions_3)
#print(floors_in_descriptions_4)
#print(floors_in_descriptions_5)
#print(floors_in_descriptions_6) # con este porcentaje podemos notar que las expresiones regulares se estan solapando

df=df.drop(["Piso"], axis=1)
df.columns

0.04898531595446296


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Ambientes',
       'Descripcion', 'Título', 'TC'],
      dtype='object')

In [20]:
#reconstruccion de la columna de ambientes en base a las columnas titulo y description
pattern = "([0-9]+)(\s)*(amb|AMB|Amb)"
#pattern = "(Mono|mono)(\s)*(amb|AMB|Amb)"
regex = re.compile(pattern)
result_title = df['Título'].apply(lambda x: regex.findall(str(x)))
ambientes_title = result_title.apply(lambda x: x[0][0] if len(x) > 0 else 0)
#ambientes_title = result_title.apply(lambda x: 1 if len(x) > 0 else 0)

df_result_title = pd.DataFrame(ambientes_title.apply(lambda x : x != 0))
df['Ambientes'][df_result_title['Título']] = ambientes_title[df_result_title['Título']]

result_description = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
ambientes_description = result_description.apply(lambda x: x[0][0] if len(x) > 0 else 0)
#ambientes_description = result_description.apply(lambda x: 1 if len(x) > 0 else 0)

df_result_description = pd.DataFrame(ambientes_description.apply(lambda x : x != 0))
df['Ambientes'][df_result_description['Descripcion']] = ambientes_description[df_result_description['Descripcion']]

#analizamos la proporcion de registros en la columna imputada
len(df["Ambientes"][df["Ambientes"].notnull()])/len(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.5706979046361986

In [21]:
df["Ambientes"][df["Ambientes"]!=0].median()

3.0

In [22]:
#construccion de la columna de amenities en base a las columnas titulo y description
pattern = "(pileta|gimnasio|quincho|parrilla|SUM)"
regex = re.compile(pattern, re.IGNORECASE)

#result_title = df['Título'].apply(lambda x: regex.findall(str(x)))
#amenities_title = result_title.apply(lambda x: '|'.join(filter(None, x)) if len(x) > 0 else None)
#df['Amenities'] = amenities_title

#los amenities obtenidos por el campo title se superponen con los obtenidos por el campo description

result_description = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
amenities_description = result_description.apply(lambda x: len(np.unique(list(map(lambda y: y.lower(), filter(None, x))))) if len(x) > 0 else None)
df['Amenities'] = amenities_description

len(df["Amenities"][df["Amenities"].notnull()])/len(df)



0.43419402738821977

In [23]:
amenities_description.median()

1.0

In [24]:
#imputamos el precio por m2 en dolares a partir del precio en m2 y el campo TC
def calculatePriceM2USD(x):
  if(np.isnan(x['Precio M2 USD']) and not np.isnan(x['Precio M2'])):
    return x['Precio M2'] / x['TC']
  else:
    return x['Precio M2']

    
df["Precio M2 USD"]=df.apply(calculatePriceM2USD, axis=1)

In [25]:
#analizamos la proporcion de registros en la columna imputada
len(df["Precio M2 USD"][df["Precio M2 USD"].notnull()])/len(df)

0.7231314964527306

In [26]:
#imputamos la superficie total a partir de los campos precio usd y precio m2 usd
def calculateSup(x):
  if(not np.isnan(x['Precio USD']) and not np.isnan(x['Precio M2 USD']) and np.isnan(x['Superficie Total'])):
    return x['Precio USD'] / x['Precio M2 USD']
  else:
    return x['Superficie Total']
    
df["Superficie Total"]=df.apply(calculateSup, axis=1)

In [27]:
#analizamos la proporcion de registros en la columna imputada
len(df["Superficie Total"][df["Superficie Total"].notnull()])/len(df)

0.8760930539514932

In [28]:
#podemos observar que las columnas partido y place_with_parent_names estan relacionadas
#la columna place_with_parent_names contiene una concatenacion de valores que describen
#a la ubicacion del inmueble donde el ultimo valor es el partido.

#imputamos la superficie total a partir de los campos precio usd y precio m2 usd
def FillPlaceNames(x):
    splited_values = x['place_with_parent_names'].split('|')[::-1]
    for value in splited_values:
        if(not (value == '')):
            return value
    return x['Partido']
    
df["Partido"]=df.apply(FillPlaceNames, axis=1)

df=df.drop(["place_with_parent_names", "geonames_id"], axis=1)
df.columns

#analizamos la proporcion de registros en la columna imputada
len(df["Partido"][df["Partido"].notnull()])/len(df)

1.0

In [29]:
# Analizamos la cantidad de nulls por filas en proporcion al total de columnas
#df_not_nulls_per_row = df.notnull().sum(axis = 1) / len(df.columns)

# filtramos por los registros que no cumplan con un cierto porcentaje de datos en sus columnas
#df_not_nulls_per_row_45_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.45])
#df_not_nulls_per_row_50_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.5])
#df_not_nulls_per_row_55_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.55])
#df_not_nulls_per_row_60_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.60])
#df_not_nulls_per_row_65_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.65])
#df_not_nulls_per_row_70_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.70])
#df_not_nulls_per_row_75_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.75])
#df_not_nulls_per_row_80_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.80])

#mostramos la cantidad porcentual de datos que
#print(' 45% de columnas vacias: ' + str(df_not_nulls_per_row_45_per) + 'u  | ' + str(df_not_nulls_per_row_45_per / len(df)) + '% registros\n',
#      '50% de columnas vacias: ' + str(df_not_nulls_per_row_50_per) + 'u  | ' +  str(df_not_nulls_per_row_50_per / len(df)) + '% registros\n',
#      '55% de columnas vacias: ' + str(df_not_nulls_per_row_55_per) + 'u | ' +  str(df_not_nulls_per_row_55_per / len(df)) + '% registros\n',
#      '60% de columnas vacias: ' + str(df_not_nulls_per_row_60_per) + 'u | ' +  str(df_not_nulls_per_row_60_per / len(df)) + '% registros\n',
#      '65% de columnas vacias: ' + str(df_not_nulls_per_row_65_per) + 'u | ' +  str(df_not_nulls_per_row_65_per / len(df)) + '% registros\n',
#      '70% de columnas vacias: ' + str(df_not_nulls_per_row_70_per) + 'u | ' +  str(df_not_nulls_per_row_70_per / len(df)) + '% registros\n',
#      '75% de columnas vacias: ' + str(df_not_nulls_per_row_75_per) + 'u | ' +  str(df_not_nulls_per_row_75_per / len(df)) + '% registros\n',
#      '80% de columnas vacias: ' + str(df_not_nulls_per_row_80_per) + 'u | ' +  str(df_not_nulls_per_row_80_per / len(df)) + '% registros')

In [30]:
#Vemos las caracteristicas generales de las distintas variables del dataset
df.describe()

,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,TC,Amenities
count,"69,670.0","69,670.0","100,810.0","106,200.0","101,313.0","87,658.0","87,658.0","100,809.0","52,633.0"
mean,-34.62620984295557,-59.26628952225032,"239,700.5769284793",698.0975802052208,133.05018112186985,"4,891.223394500322","6,912.215704519683",17.644499988455586,1.6770087207645394
std,1.980936251322784,2.2999223901611723,"391,323.8757320808","10,635.18804762383",724.3514788261274,"13,095.232796524984","28,378.644733892193",4.607770093825213e-08,0.8108591473656144
min,-54.82398489999999,-75.67893099,0.0,0.0,0.0,0.1818445492778271,1.5102040816326532,17.644498249101435,1.0
25%,-34.669065,-58.727040349999996,"89,733.88",55.0,45.0,637.5924509053813,"1,550.0",17.6445,1.0
50%,-34.597985113,-58.4801284,"145,000.0",120.0,75.0,"1,803.2852344315206","2,213.114754098361",17.6445,1.0
75%,-34.441299275,-58.395907675000004,"265,000.0",487.0,150.0,"2,926.829268292683","3,355.5486111111113",17.6445,2.0
max,4.5458426,-53.73332979999999,"46,545,445.0","3,299,521.5000000005","187,000.0","692,046.0","3,999,999.96",17.6445,5.0


In [31]:
#Obervamos si las variables estan correlacionadas
df_corr=df.corr().round(4)
df_corr

,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,TC,Amenities
Latitud,1.0,-0.2943,0.0671,0.0073,0.0081,0.095,0.0861,-0.158,0.0173
Longitud,-0.2943,1.0,-0.0261,-0.0089,-0.0095,-0.1019,-0.0878,0.1941,0.0652
Precio USD,0.0671,-0.0261,1.0,0.0724,0.1655,0.0295,0.0561,0.1051,0.122
Superficie Total,0.0073,-0.0089,0.0724,1.0,0.8475,-0.0204,-0.011,0.0137,0.0168
Superfecie Cubierta,0.0081,-0.0095,0.1655,0.8475,1.0,-0.0289,-0.0206,0.0191,0.025
Precio M2 USD,0.095,-0.1019,0.0295,-0.0204,-0.0289,1.0,0.5298,-0.2155,0.0413
Precio M2,0.0861,-0.0878,0.0561,-0.011,-0.0206,0.5298,1.0,-0.1301,0.0199
TC,-0.158,0.1941,0.1051,0.0137,0.0191,-0.2155,-0.1301,1.0,0.0204
Amenities,0.0173,0.0652,0.122,0.0168,0.025,0.0413,0.0199,0.0204,1.0


# Modelado

In [32]:
#verificamos como quedo nuestro modelo despues de la limpieza de datos
df_not_nulls= df.notnull().sum(axis=0) / len(df)
df_not_nulls.round(4).sort_values(ascending = False)

Tipo de propiedad        1.0
Partido                  1.0
Provincia                1.0
Descripcion              1.0
Título                   1.0
Superficie Total      0.8761
Superfecie Cubierta   0.8358
Precio USD            0.8316
TC                    0.8316
Precio M2 USD         0.7231
Precio M2             0.7231
Latitud               0.5747
Longitud              0.5747
Ambientes             0.5707
Amenities             0.4342
dtype: float64

In [33]:
#vemos que los valores de la columna precio m2 usd son muy variables
#def fixPrices(x):
#  if(np.isnan(x['Precio USD']) and not np.isnan(x['Superficie Total']) and x['Superficie Total'] > 0):
#    return x["Precio USD"] / x["Superficie Total"]
#  else:
#    return x["Precio M2 USD"]
#
#df["Precio M2 USD"] = df.apply(fixPrices, axis=1)

In [34]:
#filtramos el modelo por los inmuebles ubicados en capital federal
state_df = df[df["Provincia"] == 'Capital Federal']
state_df.head(3)

,Tipo de propiedad,Partido,Provincia,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,Ambientes,Descripcion,Título,TC,Amenities
0,PH,Mataderos,Capital Federal,-34.6618237,-58.5088387,"62,000.0",55.0,40.0,"1,550.0","1,550.0",2,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,17.6445,NaN
2,apartment,Mataderos,Capital Federal,-34.6522615,-58.5229825,"72,000.0",55.0,55.0,"1,309.090909090909","1,309.090909090909",2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,17.6445,NaN
3,PH,Liniers,Capital Federal,-34.6477969,-58.5164244,"95,000.0",NaN,NaN,NaN,NaN,3,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,17.6445,NaN


In [35]:
full_df = state_df[["Tipo de propiedad", "Partido", "Superfecie Cubierta", "Ambientes", "Amenities", "Precio USD"]].isnull().any(axis=1)
full_df[~full_df].size

9184

In [36]:
filtered_df = state_df[["Tipo de propiedad", "Partido", "Superfecie Cubierta", "Ambientes", "Amenities", "Precio USD", "Precio M2 USD"]][~full_df]
filtered_df

,Tipo de propiedad,Partido,Superfecie Cubierta,Ambientes,Amenities,Precio USD,Precio M2 USD
13,apartment,Palermo Soho,30.0,3,1.0,"111,700.0","3,723.333333333334"
14,apartment,Palermo Soho,31.0,1.0,1.0,"147,900.0","4,770.967741935484"
16,PH,Mataderos,98.0,4,1.0,"239,000.0","2,438.775510204081"
38,apartment,Las Cañitas,130.0,3.0,3.0,"480,000.0","3,692.307692307693"
51,house,Boedo,324.0,6,2.0,"550,000.0","1,697.5308641975312"
...,...,...,...,...,...,...,...
121113,apartment,Palermo,88.0,3,2.0,"265,000.0","3,011.3636363636365"
121120,apartment,Palermo Soho,40.0,2,3.0,"160,000.0","4,000.0"
121138,apartment,Saavedra,60.0,2,3.0,"208,000.0","3,466.666666666666"
121153,apartment,Barrio Norte,65.0,3,2.0,"165,000.0","2,538.4615384615386"


In [37]:
# verificamos que variables van a requerir de algun tipo de codificacion
# debatimos sobre si quedarnos unicamente con los registros de palermo ya que existe una gran proporcion de ellos en el dataset

print(np.unique(filtered_df["Tipo de propiedad"]))
print(np.unique(filtered_df["Partido"]))

['PH' 'apartment' 'house' 'store']
['Abasto' 'Agronomía' 'Almagro' 'Balvanera' 'Barracas' 'Barrio Norte'
 'Belgrano' 'Boca' 'Boedo' 'Caballito' 'Capital Federal'
 'Centro / Microcentro' 'Chacarita' 'Coghlan' 'Colegiales' 'Congreso'
 'Constitución' 'Flores' 'Floresta' 'Las Cañitas' 'Liniers' 'Mataderos'
 'Monserrat' 'Monte Castro' 'Nuñez' 'Once' 'Palermo' 'Palermo Chico'
 'Palermo Hollywood' 'Palermo Soho' 'Palermo Viejo' 'Parque Avellaneda'
 'Parque Centenario' 'Parque Chacabuco' 'Parque Chas' 'Parque Patricios'
 'Paternal' 'Pompeya' 'Puerto Madero' 'Recoleta' 'Retiro' 'Saavedra'
 'San Cristobal' 'San Nicolás' 'San Telmo' 'Tribunales' 'Velez Sarsfield'
 'Versalles' 'Villa Crespo' 'Villa Devoto' 'Villa General Mitre'
 'Villa Lugano' 'Villa Luro' 'Villa Ortuzar' 'Villa Pueyrredón'
 'Villa Real' 'Villa Riachuelo' 'Villa Santa Rita' 'Villa Soldati'
 'Villa Urquiza' 'Villa del Parque']


In [38]:
# Observamos que para las features seleccionadas, casi el 90% de los datos pertenece a apartments.
# Podemos quedarnos solo con los registros de apartments o buscar registros para los otros tipos de propiedades.
filtered_df['Tipo de propiedad'].value_counts()

apartment    8003
house         587
PH            568
store          26
Name: Tipo de propiedad, dtype: int64

In [39]:
#nos quedamos solo con los registros de appartments
#filtered_df = filtered_df[filtered_df["Tipo de propiedad"] == 'apartment']
#filtered_df=filtered_df.drop(["Tipo de propiedad"], axis=1)

In [40]:
#vemos que existen muchos barrios que se llaman palermo.
filtered_df['Partido'].value_counts()

Palermo            773
Caballito          771
Belgrano           768
Boedo              658
Villa Urquiza      572
                  ... 
Pompeya             11
Villa Real          10
Tribunales           9
Villa Riachuelo      2
Villa Soldati        2
Name: Partido, Length: 61, dtype: int64

In [41]:
#para evitar el solapamiento de los registros pertenecientes a alguna de las zonas de palermo, unificamos esos barrios en uno solo
filtered_df["Partido"] = filtered_df["Partido"].replace(['Palermo Chico','Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo'],'Palermo')

In [42]:
#procedemos a eliminar los outlier
#def remove_partido_outliers(df):
#  df_out=pd.DataFrame()
#  for key,subdf in df.groupby('Partido'):
#    m=np.mean(subdf["Precio USD"])
#    st=np.std(subdf["Precio USD"])
#    reduced_df=subdf[(subdf["Precio USD"]>(m-st))& (subdf["Precio USD"]<(m+st))]
#    df_out=pd.concat([df_out,reduced_df],ignore_index=True)
#  return df_out
#filtered_df=remove_partido_outliers(filtered_df)
#filtered_df

In [43]:
#procedemos a eliminar los outlier para "Precio M2 USD"
def remove_partido_outliers(df):
  df_out=pd.DataFrame()
  for key,subdf in df.groupby('Partido'):
    m=np.mean(subdf["Precio M2 USD"])
    st=np.std(subdf["Precio M2 USD"])
    reduced_df=subdf[(subdf["Precio M2 USD"]>(m-st))& (subdf["Precio M2 USD"]<(m+st))]
    df_out=pd.concat([df_out,reduced_df],ignore_index=True)
  return df_out
filtered_df=remove_partido_outliers(filtered_df)
filtered_df

,Tipo de propiedad,Partido,Superfecie Cubierta,Ambientes,Amenities,Precio USD,Precio M2 USD
0,apartment,Abasto,95.0,4,2.0,"275,000.0","2,894.736842105264"
1,apartment,Abasto,164.0,5,2.0,"440,000.0","2,682.926829268293"
2,apartment,Abasto,53.0,2,3.0,"160,000.0","3,018.867924528302"
3,PH,Abasto,68.0,3.0,1.0,"129,000.0","1,897.058823529412"
4,apartment,Abasto,87.0,4,2.0,"227,000.0","2,609.195402298851"
...,...,...,...,...,...,...,...
8110,house,Villa del Parque,20.0,6,3.0,"520,000.0","1,473.5469976479922"
8111,house,Villa del Parque,191.0,4,1.0,"400,000.0",118.69085764381738
8112,apartment,Villa del Parque,62.0,3,1.0,"163,000.0","2,629.032258064516"
8113,apartment,Villa del Parque,47.0,2,3.0,"120,000.0","2,553.1914893617018"


In [44]:
#Definimos nuestro dataframe
#Utilizamos todos los registros que hay en el dataset filtrado
df_model = filtered_df
df_model

,Tipo de propiedad,Partido,Superfecie Cubierta,Ambientes,Amenities,Precio USD,Precio M2 USD
0,apartment,Abasto,95.0,4,2.0,"275,000.0","2,894.736842105264"
1,apartment,Abasto,164.0,5,2.0,"440,000.0","2,682.926829268293"
2,apartment,Abasto,53.0,2,3.0,"160,000.0","3,018.867924528302"
3,PH,Abasto,68.0,3.0,1.0,"129,000.0","1,897.058823529412"
4,apartment,Abasto,87.0,4,2.0,"227,000.0","2,609.195402298851"
...,...,...,...,...,...,...,...
8110,house,Villa del Parque,20.0,6,3.0,"520,000.0","1,473.5469976479922"
8111,house,Villa del Parque,191.0,4,1.0,"400,000.0",118.69085764381738
8112,apartment,Villa del Parque,62.0,3,1.0,"163,000.0","2,629.032258064516"
8113,apartment,Villa del Parque,47.0,2,3.0,"120,000.0","2,553.1914893617018"


In [45]:
#definimos las variables dummies para la columna de partido
dummies_partido = pd.get_dummies(df_model['Partido'],drop_first=True)
dummies_partido

,Agronomía,Almagro,Balvanera,Barracas,Barrio Norte,Belgrano,Boca,Boedo,Caballito,Capital Federal,...,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
#definimos las variables dummies para la columna de tipo de propiedad
dummies_propiedad = pd.get_dummies(df_model['Tipo de propiedad'],drop_first=True)
dummies_propiedad

,apartment,house,store
0,1,0,0
1,1,0,0
2,1,0,0
3,0,0,0
4,1,0,0
...,...,...,...
8110,0,1,0
8111,0,1,0
8112,1,0,0
8113,1,0,0


In [47]:
df_model = pd.concat([df_model,dummies_partido, dummies_propiedad],axis=1)
df_model

,Tipo de propiedad,Partido,Superfecie Cubierta,Ambientes,Amenities,Precio USD,Precio M2 USD,Agronomía,Almagro,Balvanera,...,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque,apartment,house,store
0,apartment,Abasto,95.0,4,2.0,"275,000.0","2,894.736842105264",0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,apartment,Abasto,164.0,5,2.0,"440,000.0","2,682.926829268293",0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,apartment,Abasto,53.0,2,3.0,"160,000.0","3,018.867924528302",0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,PH,Abasto,68.0,3.0,1.0,"129,000.0","1,897.058823529412",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apartment,Abasto,87.0,4,2.0,"227,000.0","2,609.195402298851",0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8110,house,Villa del Parque,20.0,6,3.0,"520,000.0","1,473.5469976479922",0,0,0,...,0,0,0,0,0,0,1,0,1,0
8111,house,Villa del Parque,191.0,4,1.0,"400,000.0",118.69085764381738,0,0,0,...,0,0,0,0,0,0,1,0,1,0
8112,apartment,Villa del Parque,62.0,3,1.0,"163,000.0","2,629.032258064516",0,0,0,...,0,0,0,0,0,0,1,1,0,0
8113,apartment,Villa del Parque,47.0,2,3.0,"120,000.0","2,553.1914893617018",0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [48]:
df_model.columns

Index(['Tipo de propiedad', 'Partido', 'Superfecie Cubierta', 'Ambientes',
       'Amenities', 'Precio USD', 'Precio M2 USD', 'Agronomía', 'Almagro',
       'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
       'Caballito', 'Capital Federal', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda',
       'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
       'Parque Patricios', 'Paternal', 'Pompeya', 'Puerto Madero', 'Recoleta',
       'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo',
       'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro',
       'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Riachuelo',
       'Villa Santa Rita', 'Villa Soldati', 'V

In [49]:
# Armamos distintas combinaciones de features
# Superficie cubierta, ambientes, amenities, barrio(partido), tipo de propiedad
features_1 = ['Superfecie Cubierta', 'Ambientes', 'Amenities', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos',
       'Monserrat', 'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita',
       'Villa Urquiza', 'Villa del Parque', 'apartment', 'house', 'store']

# ambientes, amenities, barrio(partido), tipo de propiedad
features_2 = ['Ambientes', 'Amenities', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos',
       'Monserrat', 'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita',
       'Villa Urquiza', 'Villa del Parque', 'apartment', 'house', 'store']
       
# superficie cubierta, amenities, barrio(partido), tipo de propiedad
features_3 = ['Superfecie Cubierta', 'Amenities', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos',
       'Monserrat', 'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita',
       'Villa Urquiza', 'Villa del Parque', 'apartment', 'house', 'store']
       
# superficie cubierta, ambientes, barrio(partido), tipo de propiedad
features_4 = ['Superfecie Cubierta', 'Ambientes', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos',
       'Monserrat', 'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita',
       'Villa Urquiza', 'Villa del Parque', 'apartment', 'house', 'store']
       


#ESTAS COMBINACIONES SE TOMARON EN CUENTA A LA HORA DE LAS PRUEBAS PERO SON DE POCA IMPORTANCIA PARA EL ANALISIS YA QUE LA ELIMINACION DE CUALQUIER CONJUNTO DE VARIALES DUMMIES (TIPO DE PROPIEDAD O BARRIO(PARTIDO))
#DISMINUYEN NOTABLEMENTE LA PERFORMANCE DEL MODELO

# superficie cubierta, ambientes, amenities, tipo de propiedad
#features_5 = ['Superfecie Cubierta', 'Ambientes', 'Amenities', 'apartment', 'house', 'store']

# superficie cubierta, ambientes, amenities, barrio(partido)
#features_6 = ['Superfecie Cubierta', 'Ambientes', 'Amenities', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
#       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos',
#       'Monserrat', 'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
#       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo',
#       'Villa Devoto', 'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita',
#       'Villa Urquiza', 'Villa del Parque']

In [50]:
from sklearn.preprocessing import StandardScaler

X = df_model[features_4]
y = df_model['Precio M2 USD']

#Separamos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state = 4)

#Instanciamos el modelo
scaler = StandardScaler()

X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

linreg = LinearRegression()
#lasso = linear_model.LassoCV()
#ridge = linear_model.RidgeCV()

In [51]:
linreg.fit(X_train_scl, y_train)
y_pred = linreg.predict(X_test_scl)

X = X_train_scl
y = y_train

y_pred

array([3110.60503183, 2445.87059992, 1335.58214949, ..., 2957.17794782,
       2446.11356928, 2587.4893811 ])

In [ ]:
#Regresion lineal

scores = cross_validate(linreg, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
print(scores['train_r2'])
print("Regresion lineal" , np.average(scores['train_r2']))

#scores = cross_validate(lasso, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
#print(scores['train_r2'])
#print(np.average(scores['train_r2']))

#scores = cross_validate(ridge, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
#print(scores['train_r2'])
#print(np.average(scores['train_r2']))

[0.45485504 0.4379111  0.48276627 0.4669174  0.44224431]
Regresion lineal 0.4569388238613416


# APLICANDO LASSO

In [ ]:
#
model_lasso_cv = linear_model.LassoCV(alphas= [0.05, 0.06, 0.07, 0.08, 0.09, 0.95, 0.97, 0.1, 0.12, 0.15, 0.2, 0.3, 1.0, 10.0], 
                                   fit_intercept=True, normalize=False, cv=5, max_iter=1e4)

model_fit_lasso_cv = model_lasso_cv.fit(X_train_scl, y_train)

print(model_fit_lasso_cv.alpha_)

print(model_fit_lasso_cv.score(X_train_scl, y_train))


1.0
0.4563548018272855


In [ ]:
best_alpha = model_fit_lasso_cv.alpha_
model_lasso = linear_model.Lasso(alpha = best_alpha, fit_intercept = True, normalize = False)
model_fit_lasso = model_lasso.fit(X_train_scl, y_train)
print(model_fit_lasso.coef_)
print(model_fit_lasso.intercept_)
print(model_fit_lasso.score(X_test_scl, y_test))

[-5.39755713e+01 -8.52334860e+01 -1.15219271e+00 -6.82310335e+01
 -7.06737577e+01 -1.73548598e+01  2.00582630e+02  1.34365676e+02
 -4.97158417e+01  2.41143071e+03  1.07739294e+02 -4.11222813e+01
  5.26240115e+01 -1.63103930e+01  5.21353536e+01  2.24578688e+02
 -9.33902269e+01 -5.91596670e+01 -1.26714165e+02 -1.21230849e+01
  9.50596368e+01 -8.41587125e+00 -4.84937607e+00 -2.41275154e+02
 -4.16995405e+01  1.24961046e+02 -9.39762157e+00  3.04707424e+02
 -5.79580050e+01  3.69040552e+00 -4.36806955e+01 -1.26665094e+02
 -1.10181049e+01  8.51363346e+00 -2.03246172e+01  5.29268379e+02
  5.88149255e+01  7.16841245e+01  3.93097593e+00  1.85774170e+03
  2.50567512e+00 -0.00000000e+00  3.38086119e+01 -4.36951354e-01
 -2.17172268e+01 -2.48610080e+01 -1.22227887e+01  1.06606571e+01
 -8.05528232e+01 -4.24970848e+01  4.32307976e+00 -8.92213267e+01
 -1.82651091e+01  3.28794147e+01 -5.63926286e+00  2.77372577e+00
  3.79075295e+02 -9.30802060e+00  2.83792152e+01]
3678.917664349713
0.42158494318781004


In [ ]:
#Regresion Ridge
model_ridge_cv = linear_model.RidgeCV(alphas= [0.05, 0.06, 0.07, 0.08, 0.09, 0.95, 0.97, 0.1, 0.12, 0.15, 0.2, 0.3, 1.0, 10.0], 
                                   fit_intercept=True, normalize=False, cv=5) #, max_iter=1e4

model_fit_ridge_cv = model_ridge_cv.fit(X_train_scl, y_train)

print(model_fit_ridge_cv.alpha_)

print(model_fit_ridge_cv.score(X_train_scl, y_train))

1.0
0.4563573479890436


In [ ]:
best_alpha = model_fit_ridge_cv.alpha_
model_ridge = linear_model.Ridge(alpha = best_alpha, fit_intercept = True, normalize = False)
model_fit_ridge = model_ridge.fit(X_train_scl, y_train)
print(model_fit_ridge.coef_)
print(model_fit_ridge.intercept_)
print(model_fit_ridge.score(X_test_scl, y_test))

[-5.52253459e+01 -8.62831390e+01 -2.44843457e+00 -7.07549434e+01
 -7.26390997e+01 -1.93439473e+01  2.00130454e+02  1.32674764e+02
 -5.13463990e+01  2.40934391e+03  1.06119641e+02 -4.32389782e+01
  5.30516115e+01 -1.81931709e+01  5.21179808e+01  2.24260665e+02
 -9.51561680e+01 -6.11341004e+01 -1.29528465e+02 -1.39828198e+01
  9.54315134e+01 -1.01865242e+01 -6.91751208e+00 -2.43560835e+02
 -4.33369833e+01  1.24537202e+02 -1.06099589e+01  3.02188274e+02
 -5.93432075e+01  4.40664314e+00 -4.54840793e+01 -1.28158177e+02
 -1.30827315e+01  8.69245320e+00 -2.16267243e+01  5.28603390e+02
  5.84072599e+01  7.23348808e+01  3.35920365e+00  1.85712979e+03
  3.08973790e+00 -2.66081759e+00  3.45204632e+01 -1.96566818e+00
 -2.32412538e+01 -2.75339294e+01 -1.44302453e+01  1.13501506e+01
 -8.22153297e+01 -4.46382645e+01  4.83796457e+00 -9.09412628e+01
 -1.96160821e+01  3.32879784e+01 -8.99492289e+00  2.22274740e+00
  3.77953832e+02 -9.24012347e+00  2.95711165e+01]
3678.917664349713
0.4216082829810549


In [ ]:
#Solo los promedios (para la tabla)
scores = cross_validate(linreg, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
print("Regresion lineal" , np.average(scores['train_r2']))

scores = cross_validate(lasso, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
print("Regresion Lasso" , np.average(scores['train_r2']))

scores = cross_validate(ridge, X, y, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
print("Regresion Ridge" , np.average(scores['train_r2']))

Regresion lineal 0.4570547176346646


NameError: name 'lasso' is not defined

# Analisis Visual de los datos

In [ ]:
#Distribucion de precio en USD de la totalidad de los datos
fig = px.histogram(df, x="Precio USD", range_x=[0,1000000], title="Distribución Precio USD")
#a continuacion detallamos el tamaño de los intervalos
fig.update_traces(xbins=dict(
        start=0.0,
        end=50000000,
        size=25000,
    ))
#agregamos la media
fig.add_vline(x=239700.577, line_color="green")
fig.show()

In [ ]:
#Distribucion del precio del m2 en USD de la totalidad de los datos
fig = px.histogram(df, x="Precio M2 USD", range_x=[0,10000], title="Distribución Precio M2 USD")
#a continuacion detallamos el tamaño de los intervalos
fig.update_traces(xbins=dict(
        start=0.0,
        end=50000000,
        size=500,
    ))
#agregamos la media
fig.add_vline(x=2160.08, line_color="green")
fig.show()

In [ ]:
#Mayor correlacion con Precio del M2 y Superfecie, basicamente pxq.
df_corr_p=df_corr.loc[(np.absolute(df_corr["Precio USD"])>0.3)|(np.absolute(df_corr["Precio USD"])>0.3)][["Precio USD","Precio M2 USD"]]
df_corr_p.loc[np.absolute(df_corr_p["Precio M2 USD"])<0.3,["Precio M2 USD"]]="-"
df_corr_p

In [ ]:
#Relacion entre Superficie Total y Precio USD
fig = px.scatter(x=df["Superficie Total"], y=df["Precio USD"], trendline="ols", trendline_color_override="red",title="Correlacion Precio USD/Superficie Total", 
                 labels={
                     "x": "Superficie Total",
                     "y": "Precio USD",
                 })
fig.update_layout(yaxis_range=[50,1000000],xaxis_range=[0,1000])
fig.show()

In [ ]:
#Relacion entre Precio M2 USD y Precio USD
fig = px.scatter(x=df["Precio M2 USD"], y=df["Precio USD"], trendline="ols", trendline_color_override="red", title="Correlacion Precio USD/Precio M2 USD", 
                 labels={
                     "x": "Precio M2 USD",
                     "y": "Precio USD",
                 })
fig.update_layout(yaxis_range=[50,1000000],xaxis_range=[0,2000])
fig.show()

In [ ]:
df.sort_values(by="Precio USD",ascending=False)[["Provincia","Precio USD"]]

In [ ]:
df["Precio USD"].describe()       

In [ ]:
#Analizamos como se distribuyen las ventas geograficamente
df_plt=pd.DataFrame({"Provincia":df.Provincia.value_counts().index,
                     "Cantidad Operaciones":df.Provincia.value_counts(),
                     "Porcentaje":((df.Provincia.value_counts()/len(df))*100).round(2)
                     })

#Creamos una funcion para englobar a las provincias con participacion menor al 1% en una nueva categoria llamada "otros".
def Reclasificador(arg):
    if arg["Porcentaje"]>1:
        return arg["Provincia"]
    else:
        return "otros"
    
#Nueva columna que contiene los elementos reclasificados, si tienen participacion mayor al 1% conservan su nombre.
df_plt["Provincias_recla"]=df_plt.apply(Reclasificador, axis=1)

df_plt.head(9)

In [ ]:
fig = px.pie(df_plt, values='Porcentaje', names="Provincias_recla", color_discrete_sequence=px.colors.sequential.RdBu, title="Participación de las provincias en las de ventas")
fig.show()

In [ ]:
#Analizamos con un box plot los precios en USD en las principales provincias 
df_filtrado_prov=df.loc[((df.Provincia=="Capital Federal")|(df.Provincia=="Bs.As. G.B.A. Zona Norte")|(df.Provincia=="Bs.As. G.B.A. Zona Sur")|(df.Provincia=="Córdoba")|(df.Provincia=="Santa Fe")|(df.Provincia=="Buenos Aires Costa Atlántica")|(df.Provincia=="Buenos Aires Interior")|(df.Provincia=="Bs.As. G.B.A. Zona Oeste"))]

fig = go.Figure()

fig.add_trace(go.Box(
    x=df_filtrado_prov["Provincia"],
    y=df_filtrado_prov["Precio USD"],
    boxmean=True))
fig.add_hline(y=239700.57, line_dash="dash", line_color="green")
fig.update_layout(yaxis_range=[-10000,1000000], title="Boxplots de Precio USD por provincia")
fig.show()

In [ ]:
#Analizamos con un box plot los precios del M2 en USD en las principales provincias 
fig = go.Figure()

fig.add_trace(go.Box(
    x=df_filtrado_prov["Provincia"],
    y=df_filtrado_prov["Precio M2 USD"],
    boxmean=True))

fig.add_hline(y=2160.09, line_dash="dash", line_color="green")
fig.update_layout(yaxis_range=[-1000,6000], title="Boxplots de Precio M2 USD por provincia")

In [ ]:
#Observamos la Superfecie Total promedio de los inmuebles por Provincia
df_filtrado_prov=df_filtrado_prov.groupby(by=["Provincia"]).mean()
df_filtrado_prov.sort_values(by="Superficie Total",ascending=False)["Superficie Total"]
df_filtrado_prov.index


In [ ]:
#Superficie Total Media por Provincia
fig = px.bar(df_filtrado_prov, x=df_filtrado_prov.index, y=df_filtrado_prov["Superficie Total"], title="Tamaño medio de los inmuebles por provincia")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
#Conclusion: aunque los precios en capital son mas bajos, el m2 es notablemente mas caro comparado con barrio norte.
#Podriamos infererir que esto se debe a que los inmuebles en CABA son mas pequeños que en Barrio Norte donde en general
#son mas caros

In [ ]:
#df[["Precio USD","TC"]]

In [ ]:
#Si redondeamos todos los tc obtenidos a 2 decimales, vemos que todas las operaciones se cerraron al mismo tc
#df.TC.round(2).unique()

In [ ]:
#Conociendo este dato podemos buscar en la serie historia del tc para ver aproximadamente en que fecha se vendieron los inmuebles.
#path2=r"C:\Users\rodri\Desktop\Data Science\Datos históricos USD_ARS.csv"

#df_tc=pd.read_csv(path2)
#df_tc=df_tc[["Fecha","Apertura"]]
#df_tc["Apertura"]=df_tc["Apertura"].apply(lambda x: float(x.replace(',','.')))
#df_tc["Apertura"]=df_tc['Apertura'].round(2)
#df_tc

In [ ]:
#Vemos que la venta se efectuo en Noviembre del 2017 aprox.
#TC=17.64
#df_tc.loc[df_tc["Apertura"]==TC]